In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd
import geopandas
from shapely.geometry import (LinearRing, LineString, MultiLineString,
                              MultiPoint, MultiPolygon, Point, Polygon)
import io
# from io import StringIO
# from sqlalchemy import create_engine
from geographiclib.geodesic import Geodesic
import sqlalchemy
import numpy as np
# import app.main as m
from app import *
from shapely.geometry import (LinearRing, LineString, MultiLineString,
                              MultiPoint, MultiPolygon, Point, Polygon)
from __dis import *

# 临时

In [11]:
data = pd.read_clipboard()

In [12]:
data_use = data.groupby(by='地市').head(15)

In [13]:
data_use.to_clipboard()

# 周边mr和流量

In [51]:
data = pd.read_csv('d:/linshi2.csv',encoding = 'gbk')

In [52]:
coverage_scene = gpd.read_file('g:/1-规划/图层/全省区域规划图层20180510/全省区域规划图层20180510/全省区域规划图层20180510.TAB',encoding = 'gbk')

In [53]:
data.columns

Index(['站点名称', 'TDL规划站号（逻辑）', '规划经度', '规划纬度', '类型', '制式'], dtype='object')

In [54]:
data_p = tools_geo.add_points(data,lon='规划经度',lat='规划纬度')

In [55]:
data_p_cj = gpd.sjoin(coverage_scene,data_p,how='right')

In [56]:
data_p_cj = data_p_cj[[ '站点名称', 'TDL规划站号（逻辑）', '规划经度', '规划纬度', '类型', '制式','区域类型']]

In [57]:
res1 = data_p_cj.copy()
res1.loc[res1['区域类型'].isna(),'区域类型']='农村'
res1['区域类型'] = res1['区域类型'].replace('3A景区', '乡镇')
res1['区域类型'] = res1['区域类型'].replace('4A景区', '乡镇')
res1['区域类型'] = res1['区域类型'].replace('5A景区', '乡镇')
res1['区域类型'] = res1['区域类型'].replace('景区', '乡镇')
res1['区域类型'] = res1['区域类型'].replace('热点', '乡镇')

In [58]:
res_use = tools_geo.changjing_distance(res1, '区域类型', '制式', beishu=3)

In [59]:
res_use_buff = tools_geo.add_buffer_df(res_use, lon='规划经度', lat='规划纬度', buff_col='distance')

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [60]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191218.csv',encoding = 'gbk'))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (15,17,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
gongcan_use = gongcan.loc[(gongcan['RRU经度']>60) & (gongcan['RRU纬度']<50)]

In [62]:
gongcan_use_p = tools_geo.add_points(gongcan_use,lon='RRU经度',lat='RRU纬度')

In [63]:
res = gpd.sjoin(res_use_buff,gongcan_use_p)

In [64]:
res1= res[['站点名称', 'TDL规划站号（逻辑）', '规划经度', '规划纬度', '类型', '制式', '区域类型', 'distance','小区CGI(*)']]

In [65]:
res1.to_csv('linshi442.csv',encoding='gbk')

# 移动站点统计

In [17]:
data = pd.read_csv(open('g:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/输出数据/汇总/移动汇总.csv',encoding = 'gbk'))

In [ ]:
data_3 = tools_geo.nearest_site(data, id_name_column='小区中文名', lon='CellLongitude', lat='CellLatitude', num_min_results=3)

In [15]:
coverage_scene = gpd.read_file('g:/1-规划/图层/全省区域规划图层20180510/全省区域规划图层20180510/全省区域规划图层20180510.TAB',encoding = 'gbk')

In [18]:
data_3_p = tools_geo.add_points(data_3,lon='CellLongitude', lat='CellLatitude')

In [51]:
res = gpd.sjoin(coverage_scene,data_3_p,how='right')

In [ ]:
res.head(10)

In [52]:
res1 = res.copy()

In [53]:
res1.loc[res1['频段_900M']=='900M','频段_900M']='FDD900'
res1.loc[res1['区域类型'].isna(),'区域类型']='农村'

res1['区域类型'] = res1['区域类型'].replace('3A景区', '乡镇')
res1['区域类型'] = res1['区域类型'].replace('4A景区', '乡镇')
res1['区域类型'] = res1['区域类型'].replace('5A景区', '乡镇')
res1['区域类型'] = res1['区域类型'].replace('景区', '乡镇')
res1['区域类型'] = res1['区域类型'].replace('热点', '乡镇')

In [37]:
res1['区域类型'] = res1['区域类型'].map({'3A景区': '乡镇', '4A景区': '乡镇','5A景区':'乡镇'})

In [57]:
res_use = tools_geo.changjing_distance(res1, '区域类型', '频段_900M', beishu=1)

In [59]:
res_use.to_csv('res_linshi.csv',encoding='gbk')

# 周边针对的小区

In [65]:
data.groupby(by = 'eutrancell_cgi').sum().to_clipboard()

In [178]:
data = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191209.csv',encoding = 'gbk'))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (15,17,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [179]:
tianmen = data.loc[(data['RRU经度']>50) & (data['RRU纬度']<50) & (data['所属地区']=='武汉') & (data['归属区县']=='江汉区') & (data['方位角']>-1)]
tianmen=tianmen[['小区CGI(*)', 'RRU经度', 'RRU纬度','方位角']]
tianmen.columns = ['id','lon','lat','fw']

In [180]:
tianmen_p = app.tools.add_points(tianmen)
tianmen_p_d = tianmen_p.reset_index(drop=True).reset_index()
tianmen_p_dic = tianmen_p_d[['id','lon','lat','fw']]
dic_lonlat = tianmen_p_dic.set_index('id').T.to_dict('list')
tianmen_index = tianmen_p_d.sindex

In [181]:
cgi = '460-00-442620-7'
juli = 500
jiaodu_jiange = 30
duida = 30

In [182]:
tianmen_buff = tianmen_p.copy()
tianmen_buff = tianmen_buff.to_crs({'init': 'epsg:32650'})
tianmen_buff['geometry'] = tianmen_buff.buffer(juli)
tianmen_buff = tianmen_buff.to_crs({'init': 'epsg:4326'})

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [183]:
data_t = tianmen_buff.bounds
tianmen_buff_use = pd.concat([tianmen_buff,data_t],axis=1)
tianmen_buff_use_reindex = tianmen_buff_use.reset_index(drop=True)
tianmen_buff_use_reindex = tianmen_buff_use_reindex[['id','minx','miny','maxx','maxy']]
dic_use = tianmen_buff_use_reindex.set_index('id').T.to_dict('list')

In [184]:
data_true=list(tianmen_index.intersection(dic_use[cgi]))

In [185]:
lon,lat,fw = dic_lonlat[cgi]

In [186]:
res_data=tianmen.iloc[data_true]

In [187]:
res_data['原经度']=lon
res_data['原纬度']=lat
res_data['原方位角']=fw

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [188]:
res_data = app.tools.distance_df(res_data, '原经度', '原纬度', 'lon', 'lat')

C:\ProgramData\Anaconda3\lib\site-packages\app\mydef.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distance'] = 2*np.arcsin(np.sqrt(a))*6371*1000


In [189]:
res_data['原经度'] = pd.to_numeric(res_data['原经度'] )
res_data['原纬度'] = pd.to_numeric(res_data['原纬度'] )
res_data['lon'] = pd.to_numeric(res_data['lon'] )
res_data['lat'] = pd.to_numeric(res_data['lat'] )

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [190]:
res_data['相对方位角'] =[get_fw(lon_y,lat_y,lon,lat) for lon_y,lat_y,lon,lat in zip(res_data['原经度'], res_data['原纬度'], res_data['lon'], res_data['lat'])]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [191]:
def jiaodu(x):
    x_res = x+180
    if x_res >360:
        x_res=abs(360-x_res)
    return x_res

In [192]:
res_data['相对方位角+180'] = res_data['相对方位角'].apply(jiaodu)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [193]:
res_data['duida'] = duida

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
res_data['方位角夹角_原'] = [jiaojiao(fw_y,fw_2,duida_x) for fw_y,fw_2,duida_x in zip(res_data['原方位角'], res_data['相对方位角'], res_data['duida'])]

In [ ]:
def jiaojiao(f1,f2,f3):        
    if (abs(f1-f2)<(f3/2))
    (360-abs(f1-f2))<(f3/2):
        res = 1
    else:
        res = 0
    return res

In [ ]:
def fangwei_jiajiao():
    if abs(res_data['相对方位角']-res_data['原方位角'])>abs(res_data['相对方位角']-res_data['原方位角']):
        res = abs(res_data['相对方位角']-res_data['原方位角'])
    else:
        res = abs(res_data['相对方位角']-res_data['原方位角'])
    return res

In [38]:
data = pd.DataFrame({'shuju':[1,1,1,12,5,5,12,12,12,12,9]} )

In [48]:
data['shuju'][6]

12

In [ ]:
data = data.copy()
data_columns='shuju'
biaoji_column = '标记'
go = 2
num = 0.15
z = data[data_columns][go]
for x,y in  data.iloc[go:].iterrows():
    bizhi = abs((y[data_columns]-z)/z)
    if bizhi > num:
        data.loc[x,biaoji_column]='标记'
        z = data[data_columns][x+1]


# 结束

# 移动平均站间距

In [ ]:
data = pd.read_csv(open('g:/1-规划/评估/竞对站点评估/2019年11月/1-基础数据/移动/汇总.csv',encoding = 'gbk'))

In [ ]:
data.columns

In [ ]:
res =mt.nearest_site(data, id_name_column='小区中文名', lon='CellLongitude', lat='CellLatitude')

In [ ]:
res_p = mt.add_points(res,lon='CellLongitude',lat='CellLatitude')

In [ ]:
res_res = gpd.sjoin(coverage_scene,res_p)

In [ ]:
res_res.columns

In [ ]:
res_res1 = res_res[[ '区域类型', 'index_right',
       'level_0', 'index', '小区中文名', '网络制式(TDD/FDD)', 'CellLongitude',
       'CellLatitude', '覆盖类型', '厂家名称', 'Indoor', 'FDD900', 'FDD1800', 'F', 'D',
       'E', 'A', 'minimum_index_1', 'minimum_index_2', 'minimum_index_3',
       'distance_1', 'distance_2', 'distance_3', 'sum']]

In [ ]:
res_res1.to_clipboard()

In [ ]:
dianxin_h.info()

In [ ]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20191205.csv',encoding = 'gbk'))

In [ ]:
data = pd.read_csv('d:/linshi/yechang1.csv',encoding = 'gbk')

In [ ]:
gongcan['小区中文名'] = gongcan['小区中文名'].fillna('无')

In [ ]:
gongcan_hz = gongcan.loc[~gongcan['小区中文名'].str.contains('W-')]

In [ ]:
gongcan_use = gongcan_hz.loc[(gongcan_hz['RRU经度']>60) & (gongcan_hz['RRU纬度']<50)]

In [ ]:
gongcan_use.shape

In [ ]:
gongcan_p = nt.add_points(gongcan_use,coords=['RRU经度','RRU纬度','',''])

In [ ]:
coverage_scene = gpd.read_file('g:/1-规划/图层/全省区域规划图层20180510/全省区域规划图层20180510/全省区域规划图层20180510.TAB',encoding = 'gbk')

In [ ]:
data.columns

In [ ]:
point_data = nt.add_points(data,coords = ['规划经度','规划纬度','',''])

In [ ]:
sjoin_data_scene = gpd.sjoin(coverage_scene,point_data)

In [ ]:
data_merge = data.merge(sjoin_data_scene,how='left',on='规划物理站名',suffixes=('','_y'))

In [ ]:
data.columns

In [ ]:
data_merge = data_merge.drop([ '地市名称', '县市名称',
       '区域名称', '覆盖面积', 'geometry', 'index_right', '地市_y', '规划站号_y',
       '规划经度_y', '规划纬度_y', '基站类型_y', '可研工期_y'],axis=1)

In [ ]:
sjoin_data_scene = data_merge.copy()

In [ ]:
sjoin_data_scene['规划站号'].fillna('无',inplace = True)

In [ ]:
sjoin_data_scene.loc[sjoin_data_scene['规划站号'].str.contains('_G_'),'制式'] = 'FDD900'

In [ ]:
sjoin_data_scene_use = sjoin_data_scene[['地市', '规划站号', '规划物理站名', '规划经度', '规划纬度', '基站类型', '可研工期','制式','区域类型']]

In [ ]:
sjoin_data_scene_use.loc[sjoin_data_scene_use['区域类型'].isin(['4A景区','5A景区','热点']),'区域类型'] ='农村'

In [ ]:
sjoin_data_scene_use.groupby(by = '区域类型').count()

In [ ]:
sjoin_data_scene_use.loc[sjoin_data_scene_use['区域类型'].isna(),'区域类型']='农村'

In [ ]:
sjoin_data_scene_use = mt.changjing_distance(sjoin_data_scene_use, '区域类型', '制式', beishu=4)

In [ ]:
point_sjoin_data_scene_use = nt.add_buffer_df(sjoin_data_scene_use, lon='规划经度', lat='规划纬度', buff_col='distance')

In [ ]:
res_data = gpd.sjoin(point_sjoin_data_scene_use,gongcan_p)

In [ ]:
res_data = res_data[['规划物理站名',  '小区CGI(*)']]

In [ ]:
res_data.to_clipboard()

# 竞对站点数量统计

In [ ]:
dianxin = pd.read_excel('g:/1-规划/评估/竞对站点评估/2019年11月/2-处理过程/电信整理汇总.xlsx',encoding = 'gbk',sheet_name='Sheet1')
liantong = pd.read_excel('g:/1-规划/评估/竞对站点评估/2019年11月/2-处理过程/联通汇总.xlsx',encoding = 'gbk',sheet_name='Sheet1')

dianxin_h = dianxin.loc[dianxin['室内外']=='室外']
liantong_h = liantong.loc[liantong['室内外']=='室外']

hubei = gpd.read_file('g:/1-规划/图层/湖北省地市-边界-边界/全省地市边界.TAB',encoding = 'gbk')
hubei = hubei.to_crs({'init':'epsg:4326'})

dianxin_h = dianxin_h.astype(str)

dianxin_h.info()

dianxin_h['经度'] = dianxin_h['经度'].str.strip()
dianxin_h['纬度'] = dianxin_h['纬度'].str.strip()
dianxin_h['纬度'] = dianxin_h['纬度'].str.replace('?','')
dianxin_h['经度'] = pd.to_numeric(dianxin_h['经度'])
dianxin_h['纬度'] = pd.to_numeric(dianxin_h['纬度'])

dianxin['经度'] = dianxin['经度'].str.strip()
dianxin['纬度'] = dianxin['纬度'].str.strip()
# liantong['经度'] = liantong['经度'].str.strip()
# liantong['纬度'] = liantong['纬度'].str.strip()

dianxin_use = dianxin_h.loc[(dianxin_h['经度']>60) & (dianxin_h['纬度']<60)]
liantong_use = liantong_h.loc[(liantong_h['经度']>60 & (liantong_h['纬度']<60))]

dianxin_p = mt.add_points(dianxin_use ,lon='经度',lat='纬度')
liantong_p = mt.add_points(liantong_use , lon='经度',lat='纬度')

dianxin_res = gpd.sjoin(hubei,dianxin_p)
liantong_res = gpd.sjoin(hubei,liantong_p)

dianxin_res.drop(['geometry','index_right'],axis=1,inplace = True)
liantong_res.drop(['geometry','index_right'],axis=1,inplace = True)

dianxin_res['kkk'] = dianxin_res['kkk'].str.replace('市','')
liantong_res['kkk'] = liantong_res['kkk'].str.replace('市','')

dianxin_res_groupby = dianxin_res.groupby(by = '900-1800-2400')
liantong_res_groupby = liantong_res.groupby(by = '900-1800-2400')

dishi = dianxin_res.groupby(by = 'kkk').agg({'地市':'count'})
dishi.columns = ['电信小区']
liantong_linshi_1=liantong_res.groupby(by = 'kkk').agg({'地市':'count'})
liantong_linshi_1.columns = ['联通小区']
dishi['联通小区']=0
dishi['联通小区'].update(liantong_linshi_1['联通小区'])

dianxin_res_qc1 = dianxin_res.drop_duplicates('基站名')
dianxin_linshi_2 = dianxin_res_qc1.groupby(by = ['kkk']).agg({'kkk':'count'})
dianxin_linshi_2.columns = ['电信总站址']
dishi['电信总站址'] = 0
dishi['电信总站址'].update(dianxin_linshi_2['电信总站址'])

liantong__res_qc1 = liantong_res.drop_duplicates('基站名')
liantong_linshi_2 = liantong__res_qc1.groupby(by = ['kkk']).agg({'kkk':'count'})
liantong_linshi_2.columns = ['联通总站址']
dishi['联通总站址'] = 0
dishi['联通总站址'].update(liantong_linshi_2['联通总站址'])

for name,dianxin_t in dianxin_res_groupby:
    dianxin_res_qc = dianxin_t.drop_duplicates('基站名')
    dianxin_linshi_2 = dianxin_res_qc.groupby(by = ['kkk']).agg({'kkk':'count'})
    dianxin_linshi_2.columns = [name+'电信总站址']
    dishi[name+'电信总站址'] = 0
    dishi[name+'电信总站址'].update(dianxin_linshi_2[name+'电信总站址'])

for name,liantong_t in liantong_res_groupby:
    liantong_res_qc = liantong_t.drop_duplicates('基站名')
    liantong_linshi_2 = liantong_res_qc.groupby(by = ['kkk']).agg({'kkk':'count'})
    liantong_linshi_2.columns = [name+'联通总站址']
    dishi[name+'联通总站址'] = 0
    dishi[name+'联通总站址'].update(liantong_linshi_2[name+'联通总站址'])

a= dishi.sum()
a.name = '总计'
dishi =dishi.append(a)

dishi.to_clipboard()

# 结束

# 张璇统计区域内小区

In [ ]:
f = mt.file_name_paths(path = 'C:/Users/Administrator/Desktop/zhangxuan/')

In [ ]:
data_total = pd.DataFrame()
for name in f:
    g_data = gpd.read_file(name)
    g_data['geometry'] = g_data['geometry'].apply(lambda x: Polygon(x.coords[:]))
    res = gpd.sjoin(g_data,gongcan_p)
    res1 = res[[ 'Name','小区中文名', '小区CGI(*)','RRU经度', 'RRU纬度', '方位角','天线挂高', '机械下倾角', '电子下倾角']]
    res1['场景'] = name.split('.')[0].split('/')[-1]
    data_total = data_total.append(res1)
    print(name,res1.shape)

In [ ]:
data_total.to_csv('zhangxuan.csv',encoding = 'gbk')

In [ ]:
g_data.plot()

In [ ]:
g_data

In [ ]:
print(type(g_data['geometry'][0]))

In [ ]:
print(g_data['geometry'][0])

In [ ]:
Polygon(a)

# 经纬度列表转成图层

In [ ]:
data = pd.read_csv(open('D:/UP/副本湖北2469高价值居民区0724.csv',encoding='gbk'))

In [ ]:
def str_to_Polygoen(a):
    list=[[float(b) for b in x.split(',')] for x in a.split(';')]
    return Polygon(list)

In [ ]:
data['geometry']=data['wgs_shape'].apply(lambda x:str_to_Polygoen(x))

In [ ]:
gdata=gpd.GeoDataFrame(data,crs={'init': 'epsg:4326'},geometry='geometry')

In [ ]:
gdata.columns = ['id2', 'id', 'province', 'city', 'name', 'adcode', 'district',
       'address', 'longtitude', 'latitude', 'type', 'typecode', 'classify',
       'area', 'wgs_long', 'wgs_lat', 'wgs_shape', 'keywords', 'create_time',
       'beizhu', '集团判定无覆盖居民区', 'geometry']

In [ ]:
gdata.shape

In [ ]:
m.to_file(gdata,outfile='shangjun.geojson')

In [ ]:
plt.plot(x)

In [ ]:
gdata.plot(1)

# ^^^结束